Hi to all! 

From the very beginning of this competition it was clear that when using StratifiedKFold we get very different results on each KFold. 
For example, according to my observations in my model (https://www.kaggle.com/aikhmelnytskyy/resnet-wavenet-my-best-single-model-ensemble) val_lwlrap at 3 KFold is 0.88-0.89 and at two 0.85-0.86. And the results on the public date network range from 0.830 to 850. To solve this problem, I decided to train 10 identical models, but with different SEED parameters. That each KFold got different pieces from a training set. The https://www.kaggle.com/aikhmelnytskyy/rainforest-connection-species-audio-detectionv1 dataset includes all 50 models (5 models for each SEED value). The training code is published in version 3 of this notebook: https://www.kaggle.com/aikhmelnytskyy/resnet-tpu-on-colab-and-kaggle. I ran all the code on Google Colab in parallel, so the code is not very elegant, but if you want you can improve it. This model is different from the one I published in this notebook: https://www.kaggle.com/aikhmelnytskyy/resnet-wavenet-my-best-single-model-ensemble. The reason is the banal possibility of Google Colab is smaller than in the kaggle notebook, so I had to reduce the number of parameters in the models.

Now, after training 50 models, I plan to average them. In my head my ideas are like some Bagging option. Correct me in the comments if I'm wrong

For the experiment in version 1 of this notebook, I average the best models from each SEED parameter, and in version 2 the worst models. Version 3 is the average of all models.

In version 4, I average only the 5 best models. I hope you understand that the result of version 3 is the most credible.

Version 4 showed why you should not rely only on the best validation models :-)
Good luck to all!!!

In version 8, I tried to build an ensemble of this model with the "best" public ensemble, the result on the one hand has not changed, but on the other hand has not deteriorated, which is good. I had free attempts so I tried a few more simple ensembles in versions 9,10,11. I hope you have already understood that this ensemble may show good results on a private test set, but it is still extremely unreliable even by the standards of public notebooks.

Version 1 - 0.842

Version 2 - 0.835

Version 3 - 0.840

Version 4 - 0.828

Version 8,9,10,11 - ensemble

Version 15 - 0.878 - An ensemble with my new best model. How to get this model I described in these discussions: https://www.kaggle.com/c/rfcx-species-audio-detection/discussion/217301

Auxiliary functions

In [ ]:
from kaggle_datasets import KaggleDatasets


import pandas as pd

def read_train_csv(n_trains,path):
    base_df=pd.DataFrame()
    for i in range(n_trains):   
        
        base_df=base_df.append(pd.read_csv(path+f'train_n_{i}.csv'), ignore_index = True) 
    return base_df

def find_best_train_csv(n_trains,path,n_bests):
    base_df=pd.DataFrame()
    for i in range(n_trains):   
        df=pd.read_csv(path+f'train_n_{i}.csv')                
        base_df=base_df.append(df.nlargest(n_bests, ['best_score']) , ignore_index = True) 
    return base_df

def find_worst_train_csv(n_trains,path,n_worst):
    base_df=pd.DataFrame()
    for i in range(n_trains):   
        df=pd.read_csv(path+f'train_n_{i}.csv')                
        base_df=base_df.append(df.nsmallest(n_worst, ['best_score']) , ignore_index = True) 
    return base_df

def average_csv(csv_list):
    BLEND=pd.read_csv('../input/resnet-wavenet-my-best-single-model-ensemble/submission.csv')
    
    BLEND.iloc[:,1:]=pd.read_csv(csv_list[0])
    for sub_csv in csv_list[1:]:
        
        BLEND.iloc[:,1:]+=pd.read_csv(sub_csv)
        
    BLEND.iloc[:,1:]=BLEND.iloc[:,1:]/len(csv_list)   
    return BLEND
    
    
def create_csv_list(df,path):
    df['CSV'] = path + df['CSV'].astype(str)
    return df["CSV"].tolist()


We average all models

In [ ]:
print (KaggleDatasets().get_gcs_path('rfcx-species-audio-detection'))

n_trains=10
path='../input/rainforest-connection-species-audio-detectionv1/KAggle/'

train_data=read_train_csv(n_trains,path)
print(train_data)
BLEND_all=create_csv_list(train_data,path)
BLEND_all=average_csv(BLEND_all)
print(BLEND_all)


BLEND_all.to_csv('submission.csv',index=False)


Now let's try an ensemble of public notebooks.

Let's be careful overfitting

In [ ]:
sub2 = pd.read_csv('../input/automl-inference-audio-detection-soliset/submission.csv')#0.876
sub3 = pd.read_csv('../input/fork-of-bagging-rainforest/submission.csv')#0.856
BLEND_all.iloc[:,1:]=(BLEND_all.iloc[:,1:]*0.30+sub3.iloc[:,1:]*0.70)*0.40+sub2.iloc[:,1:]*0.60/5
BLEND_all.to_csv('submission_ensemble.csv',index=False)

We average the models with the best val_lwlrap in each SEED

In [ ]:
train_best=find_best_train_csv(n_trains,path,3)
print(train_best)

BLEND_best=create_csv_list(train_best,path)
BLEND_best=average_csv(BLEND_best)
print(BLEND_best)
BLEND_best.to_csv('submission_best.csv',index=False)

* We average the models with the worst val_lwlrap in each SEED

In [ ]:
train_worst=find_worst_train_csv(n_trains,path,2)
print(train_worst)

BLEND_worst=create_csv_list(train_worst,path)
BLEND_worst=average_csv(BLEND_worst)
print(BLEND_worst)
BLEND_worst.to_csv('submission_worst.csv',index=False)

We average the models with the best val_lwlrap

In [ ]:
train_data=read_train_csv(n_trains,path)
print(train_data)
BLEND_all=create_csv_list(train_data.nlargest(5, ['best_score']),path)
BLEND_all=average_csv(BLEND_all)
print(BLEND_all)
BLEND_all.to_csv('submission_best_5.csv',index=False)

Add Post Process from here: https://www.kaggle.com/cdeotte/rainforest-post-process-lb-0-970

In [ ]:
MODE = 3

# LOAD SUBMISSION
import pandas as pd, numpy as np
FUDGE = 2.0
FILE = 'submission.csv'
df = BLEND_all
for k in range(24):
    df.iloc[:,1+k] -= df.iloc[:,1+k].min()
    df.iloc[:,1+k] /= df.iloc[:,1+k].max()

# CONVERT PROBS TO ODDS, APPLY MULTIPLIER, CONVERT BACK TO PROBS
def scale(probs, factor):
    probs = probs.copy()
    idx = np.where(probs!=1)[0]
    odds = factor * probs[idx] / (1-probs[idx])
    probs[idx] =  odds/(1+odds)
    return probs

# DIFFERENT DISTRIBUTIONS
d1 = df.iloc[:,1:].mean().values
d2 = np.array([113, 204, 44, 923, 53, 41, 3, 213, 44, 23, 26, 149, 255,  
    14, 123, 222, 46, 6, 474, 4, 17, 18, 23, 72])/1000.

for k in range(24):
    if MODE==1: d = FUDGE
    if MODE==2: d = d1[k]/(1-d1[k])
    if MODE==3: s = d2[k] / d1[k]
    else: s = (d2[k]/(1-d2[k]))/d
    df.iloc[:,k+1] = scale(df.iloc[:,k+1].values,s)
    
sub4 = pd.read_csv('../input/lb0984rainforestcomparativemethodpartb/submission.csv')#0.984   
    
BLEND_all.iloc[:,1:]=df.iloc[:,1:]*0.10+sub4.iloc[:,1:]*0.90
df.to_csv('submission_with_pp.csv',index=False)




